In [ ]:
using CSV, Tables
using JuMP
using Gurobi

import Pkg; Pkg.add("JSON3")
using JSON3

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.10/Project.toml`
  No Changes to `~/.julia/environments/v1.10/Manifest.toml`


In [4]:
const GRB_ENV = Gurobi.Env(output_flag=1);

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-08


## load in data

In [5]:
all_classes = open("data/all_classes_info.json", "r") do file 
    JSON3.read(file)
end

R = CSV.File("data/ratings.csv",header=0) |> Tables.matrix;
H = CSV.File("data/hours.csv",header=0) |> Tables.matrix;
U = CSV.File("data/units.csv",header=0) |> Tables.matrix;

vars = CSV.File("data/variables.csv",header=0) |> Tables.matrix;
n = vars[1];
S = vars[2];

In [13]:
all_classes[1]

JSON3.Object{Base.CodeUnits{UInt8, String}, SubArray{UInt64, 1, Vector{UInt64}, Tuple{UnitRange{Int64}}, true}} with 28 entries:
  :rating              => 5.4
  :gir_attribute       => "REST"
  :has_final           => false
  :description         => "Presents engineering problems in a computational set…
  :offered_fall        => false
  :offered_spring      => true
  :meets_with_subjects => ["1.001"]
  :instructors         => ["J. Williams"]
  :out_of_class_hours  => 7.99
  :total_units         => 12
  :related_subjects    => ["2.156", "1.205", "1.C51", "1.000", "1.631", "1.063"…
  :pdf_option          => false
  :in_class_hours      => 5.7
  :is_half_class       => false
  :level               => "U"
  :prerequisites       => "GIR:CAL1"
  :subject_id          => "1.00"
  :title               => "Engineering Computation and Data Science"
  :lab_units           => 2
  :design_units        => 0
  :public              => true
  :offered_summer      => false
  :lecture_units       => 3
  :

In [16]:
findfirst(x -> x[:subject_id] == "8.01", all_classes)

3856

In [59]:
chem_gir_ids = ["3.091", "5.111", "5.112"]
phys_1_gir_ids = ["8.01", "8.011", "8.012", "8.01L"]
phys_2_gir_ids = ["8.02", "8.021", "8.022"]
math_1_gir_ids = ["18.01", "18.01A"]
math_2_gir_ids = ["18.02", "18.02A", "18.022"]
bio_gir_ids = ["7.012", "7.013", "7.014", "7.015", "7.016"]
gir_ids = [chem_gir_ids, phys_1_gir_ids, phys_2_gir_ids, math_1_gir_ids, math_2_gir_ids, bio_gir_ids]
gir_subject_types = [[findfirst(x -> x[:subject_id] == gir_id, all_classes) for gir_id in gir_type] for gir_type in gir_ids]

6-element Vector{Vector{Int64}}:
 [2962, 3303, 3304]
 [3856, 3857, 3858, 3859]
 [3860, 3861, 3862]
 [1536, 1537]
 [1538, 1540, 1539]
 [3752, 3753, 3754, 3755, 3756]

## optimize

In [78]:
model = Model(() -> Gurobi.Optimizer(GRB_ENV))

α = 0.8

# @variable(model, X[1:n,1:s] >=0)
@variable(model, X[1:n, 1:S], Bin)
@objective(model, Min, sum(α*H[i]*X[i,s] - (1-α)*R[i]*X[i,s] for i=1:n,s=1:S));

##
# CONSTRAINTS
##

# full-time; at least 36 units per semester
@constraint(model, [s in 1:S], sum(U[i]*X[i, s] for i=1:n) >= 36);

# wellbeing: no more than 112 hours per week
@constraint(model, [s in 1:S], sum(H[i]*X[i, s] for i=1:n) <= 112);

# not repeatable: cannot take a class again
@constraint(model, [i in 1:n], sum(X[i, t] for t=1:S) <= 1)

# GIR constraints
@constraint(model, [gir_type in gir_subject_types], sum(X[gir_ind, s] for gir_ind in gir_type, s in 1:S) == 1)

# taken correct semester
@constraint(model, [s in 1:2:S, i in 1:n], X[i, s] <= all_classes[i][:offered_fall])
@constraint(model, [s in 2:2:S, i in 1:n], X[i, s] <= all_classes[i][:offered_spring])

# 2 CI-H

# 8 HASS

# 1 of each HASS-H, A, S


2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, 2:2:8
    Dimension 2, Base.OneTo(4703)
And data, a 4×4703 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 X[1,2] ≤ 1  X[2,2] ≤ 0  X[3,2] ≤ 1  …  X[4702,2] ≤ 0  X[4703,2] ≤ 1
 X[1,4] ≤ 1  X[2,4] ≤ 0  X[3,4] ≤ 1     X[4702,4] ≤ 0  X[4703,4] ≤ 1
 X[1,6] ≤ 1  X[2,6] ≤ 0  X[3,6] ≤ 1     X[4702,6] ≤ 0  X[4703,6] ≤ 1
 X[1,8] ≤ 1  X[2,8] ≤ 0  X[3,8] ≤ 1     X[4702,8] ≤ 0  X[4703,8] ≤ 1

In [79]:
optimize!(model)

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[x86] - Darwin 23.6.0 23G93)

CPU model: Intel(R) Core(TM) i5-8257U CPU @ 1.40GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 42349 rows, 37624 columns and 146392 nonzeros
Model fingerprint: 0x2b082468
Variable types: 0 continuous, 37624 integer (37624 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [2e-02, 6e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 177.0920000
Presolve removed 38268 rows and 19808 columns
Presolve time: 0.17s
Presolved: 4081 rows, 17816 columns, 53432 nonzeros
Found heuristic solution: objective 132.5380000
Variable types: 0 continuous, 17816 integer (17816 binary)
Found heuristic solution: objective 112.3000000

Root relaxation: objective 6.148400e+01, 131 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds 

In [80]:
taken_classes_sem = [[ind for (ind, val) in enumerate(sem) if val == 1] for sem in eachcol(value.(X))]

8-element Vector{Vector{Int64}}:
 [296, 1180, 1539, 2021, 2600, 4511]
 [714, 816, 3857]
 [725, 1324, 1914, 3861, 4141]
 [1350, 1536, 3984]
 [3755, 4160, 4492]
 [100, 144, 974, 1025]
 [294, 1946, 1953, 2885, 2962, 3017, 4557]
 [978, 1666, 2919, 3337, 4347]

## Interpreter

In [75]:
function interpret_course_nums(classes_id_per_sem)
    course_nums = [[all_classes[class_ind][:subject_id] for class_ind in sem] for sem in classes_id_per_sem]
    return course_nums
end

function interpret_fall_courses(classes_id_per_sem)
    courses_fall = [[all_classes[class_ind][:offered_fall] for class_ind in sem] for sem in classes_id_per_sem]
    return courses_fall
end

function interpret_spring_courses(classes_id_per_sem)
    courses_spring = [[all_classes[class_ind][:offered_spring] for class_ind in sem] for sem in classes_id_per_sem]
    return courses_spring
end

interpret_spring_courses (generic function with 1 method)

In [81]:
interpret_course_nums(taken_classes_sem)

8-element Vector{Vector{String}}:
 ["10.991", "15.839", "18.022", "20.S900", "21M.622", "MS.101"]
 ["12.834", "14.399", "8.011"]
 ["12.900", "16.634", "2.991", "8.021", "CMS.627"]
 ["16.84", "18.01", "9.271"]
 ["7.015", "CMS.827", "MAS.921"]
 ["1.267", "1.713", "15.323", "15.3941"]
 ["10.960", "20.001", "20.053", "24.93", "3.091", "3.903", "SP.257"]
 ["15.335", "18.727", "3.006", "5.561", "HST.533"]

In [82]:
interpret_fall_courses(taken_classes_sem)

8-element Vector{Vector{Bool}}:
 [1, 1, 1, 1, 1, 1]
 [0, 0, 0]
 [1, 1, 1, 1, 1]
 [0, 1, 0]
 [1, 1, 1]
 [0, 0, 0, 0]
 [1, 1, 1, 1, 1, 1, 1]
 [0, 0, 1, 0, 0]

In [83]:
interpret_spring_courses(taken_classes_sem)

8-element Vector{Vector{Bool}}:
 [0, 1, 0, 1, 0, 1]
 [1, 1, 1]
 [0, 0, 0, 0, 0]
 [1, 1, 1]
 [0, 0, 0]
 [1, 1, 1, 1]
 [1, 0, 1, 0, 1, 1, 0]
 [1, 1, 1, 1, 1]